<a href="https://colab.research.google.com/github/sandhyaananthan121/OpenCV_Features/blob/main/Feature_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Sandhya Ananthan UFID: 1095-3582
# Programming Assignment -2 CAP 5416
# Feature 2 Implementation

import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim

# Define hyperparameters for the training and optimization:
n_epochs = 10
batch_size_train = 64
batch_size_test = 1024
learning_rate = 0.01
momentum = 0.5
log_interval = 10

# We will set a fixed random seed to ensure reproducibility of training.
torch.manual_seed(1)

def generate_training_transforms():
  """
  Returns a transformation object which 1) converts images to tensors and 2) normalizes image pixel statistics.
  """
  return transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)) ])

def generate_testing_transforms():
  """
  Returns a transformation object which 1) converts images to tensors and 2) normalizes image pixel statistics.
  """
  return transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (1.0, 1.0, 1.0)) ])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=generate_training_transforms())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size_train,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=generate_testing_transforms())
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size_test,
                                         shuffle=False, num_workers=2)

example = next(iter(trainloader))
type(example)

print("Example is of length {}".format(len(example)))
print("Type of first argument is {}".format(type(example[0])))
print("Type of second argument is {}".format(type(example[1])))

print("First argument has shape: {}".format(example[0].shape))
print("Second argument has shape {}".format(example[1].shape))

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32*32*3, 10)

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        return x

network = Net() # Initialize our model defined above.
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum) # Initialize our optimizer


def train_epoch():
    network.train()  # Put network into "training" mode
    # Assign Extra Variables
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for data, target in trainloader:
        optimizer.zero_grad()  # Make sure all parameters have an initial gradient of 0
        output = network(data)  # Forward pass through the network
        loss = F.cross_entropy(output, target)  # Compute the loss
        loss.backward()  # Backward pass assigns a gradient to every parameter
        optimizer.step()  # The optimizer applies the weight update equation

        # Here, We use the new variable to keep track of average loss and accuracy, first we get total_loss using the below function
        total_loss += loss.item() * data.size(0)

        # As Asked in feature, I used softmax for probablities and dim is the dimention, F.softmax is a torch function.
        probabilities = F.softmax(output, dim=1)
        # Same, told to use argmax for predicted_class. torch.argmax is a torch class function, dim is the dimentions.
        predicted_classes = torch.argmax(probabilities, dim=1)

        # Calculate accuracy
        correct_predictions += (predicted_classes == target).sum().item()
        total_samples += data.size(0)

    average_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples
    return average_loss, accuracy

# Till Here, the code is kept as it is, we add new test_epoch, where we use torch.no_grad() function for
def test_epoch():
    network.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    # This is a new with function we add no_grad from [2] in a while loop, this makes sure this dataset is not used for training.
    with torch.no_grad():
      # The rest remains the same.
        for data, target in testloader:
            output = network(data)  # Forward pass through the network
            loss = F.cross_entropy(output, target)  # Compute the loss

            # Here, We use the new variable to keep track of average loss and accuracy, first we get total_loss using the below function
            total_loss += loss.item() * data.size(0)

            # As Asked in feature, I used softmax for probablities and dim is the dimention, F.softmax is a torch function.
            probabilities = F.softmax(output, dim=1)
            # Same, told to use argmax for predicted_class. torch.argmax is a torch class function, dim is the dimentions.
            predicted_classes = torch.argmax(probabilities, dim=1)

            # Calculate accuracy
            correct_predictions += (predicted_classes == target).sum().item()
            total_samples += data.size(0)

    average_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples
    return average_loss, accuracy

for epoch in range(n_epochs):
  # Prints Feature 1 output
    avg_loss, accuracy = train_epoch()
    print(f"Epoch {epoch + 1} of {n_epochs}, Train - Avg Loss: {avg_loss:.4f}, Accuracy: {accuracy * 100:.2f}%")
  # Print the test values for comparision - Feature 2 needed
    test_avg_loss, test_accuracy = test_epoch()
    print(f"Epoch {epoch + 1} of {n_epochs}, Test - Avg Loss: {test_avg_loss:.4f}, Accuracy: {test_accuracy * 100:.2f}%")


100%|██████████| 170498071/170498071 [00:05<00:00, 28439508.51it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Example is of length 2
Type of first argument is <class 'torch.Tensor'>
Type of second argument is <class 'torch.Tensor'>
First argument has shape: torch.Size([64, 3, 32, 32])
Second argument has shape torch.Size([64])
Epoch 1 of 10, Train - Avg Loss: 1.8639, Accuracy: 36.01%
Epoch 1 of 10, Test - Avg Loss: 1.7877, Accuracy: 38.82%
Epoch 2 of 10, Train - Avg Loss: 1.7669, Accuracy: 39.75%
Epoch 2 of 10, Test - Avg Loss: 1.7521, Accuracy: 40.10%
Epoch 3 of 10, Train - Avg Loss: 1.7384, Accuracy: 40.69%
Epoch 3 of 10, Test - Avg Loss: 1.7383, Accuracy: 40.42%
Epoch 4 of 10, Train - Avg Loss: 1.7212, Accuracy: 41.31%
Epoch 4 of 10, Test - Avg Loss: 1.7276, Accuracy: 40.54%
Epoch 5 of 10, Train - Avg Loss: 1.7102, Accuracy: 41.80%
Epoch 5 of 10, Test - Avg Loss: 1.7246, Accuracy: 40.27%
Epoch 6 of 10, Train - Avg Loss: 1.7017, Accuracy: 42.06%
Epoch 6 of 10, Test - Avg Loss: 1.7221, Accuracy: 40.75%
Ep